In [18]:
from langchain.llms.cohere import Cohere
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Define Cohere API client
cohere_llm = Cohere(cohere_api_key="9rZaWYoS4JsQMcOMZjozd66X0DqbStcRVe8LimVQ", model="command-r-plus")

# Sample topics for testing
sample_topics = [
    "Inheritance and Polymorphism in C++",
    "Encapsulation and Abstraction in C++",
    "Virtual Functions and Abstract Classes",
    "STL: Vectors and Iterators",
    "Exception Handling in C++",
    "Templates and Generic Programming",
    "Operator Overloading",
    "File Handling in C++",
    "Multithreading in C++",
    "Design Patterns: Singleton and Factory"
]

# Sample student solutions for testing
sample_solutions = [
    """
    #include <iostream>
    using namespace std;
    class Animal {
    public:
        virtual void sound() {
            cout << "Animal makes a sound" << endl;
        }
    };
    class Dog : public Animal {
    public:
        void sound() override {
            cout << "Dog barks" << endl;
        }
    };
    int main() {
        Animal* obj = new Dog();
        obj->sound();
        delete obj;
        return 0;
    }
    """,
    """
    #include <iostream>
    using namespace std;
    class BankAccount {
    private:
        double balance;
    public:
        BankAccount(double initialBalance) : balance(initialBalance) {}
        void deposit(double amount) {
            if (amount > 0) {
                balance += amount;
            }
        }
        double getBalance() const {
            return balance;
        }
    };
    int main() {
        BankAccount account(1000);
        account.deposit(500);
        cout << "Balance: " << account.getBalance() << endl;
        return 0;
    }
    """,
    """
    #include <iostream>
    using namespace std;
    class Shape {
    public:
        virtual double area() const = 0;  // Pure virtual function
        virtual void display() const = 0;
    };
    class Circle : public Shape {
    private:
        double radius;
    public:
        Circle(double r) : radius(r) {}
        double area() const override {
            return 3.14159 * radius * radius;
        }
        void display() const override {
            cout << "Circle with area: " << area() << endl;
        }
    };
    int main() {
        Shape* shape = new Circle(5);
        shape->display();
        delete shape;
        return 0;
    }
    """,
    """
    #include <iostream>
    #include <vector>
    using namespace std;
    int main() {
        vector<int> numbers = {1, 2, 3, 4, 5};
        for (auto it = numbers.begin(); it != numbers.end(); ++it) {
            cout << *it << " ";
        }
        cout << endl;
        return 0;
    }
    """,
    """
    #include <iostream>
    using namespace std;
    int main() {
        try {
            int a = 10, b = 0;
            if (b == 0) {
                throw runtime_error("Division by zero!");
            }
            cout << "Result: " << a / b << endl;
        } catch (const exception& e) {
            cout << "Error: " << e.what() << endl;
        }
        return 0;
    }
    """,
    """
    #include <iostream>
    using namespace std;
    template <typename T>
    T add(T a, T b) {
        return a + b;
    }
    int main() {
        cout << "Sum of integers: " << add<int>(5, 10) << endl;
        cout << "Sum of doubles: " << add<double>(3.5, 7.8) << endl;
        return 0;
    }
    """,
    """
    #include <iostream>
    using namespace std;
    class Complex {
    private:
        double real, imag;
    public:
        Complex(double r, double i) : real(r), imag(i) {}
        Complex operator+(const Complex& other) const {
            return Complex(real + other.real, imag + other.imag);
        }
        void display() const {
            cout << real << " + " << imag << "i" << endl;
        }
    };
    int main() {
        Complex c1(3, 4), c2(1, 2);
        Complex c3 = c1 + c2;
        c3.display();
        return 0;
    }
    """,
    """
    #include <iostream>
    #include <fstream>
    using namespace std;
    int main() {
        ofstream outFile("example.txt");
        outFile << "Hello, file!" << endl;
        outFile.close();
        ifstream inFile("example.txt");
        string content;
        getline(inFile, content);
        cout << "Read from file: " << content << endl;
        inFile.close();
        return 0;
    }
    """,
    """
    #include <iostream>
    #include <thread>
    using namespace std;
    void printNumbers() {
        for (int i = 1; i <= 5; ++i) {
            cout << i << " ";
        }
        cout << endl;
    }
    int main() {
        thread t1(printNumbers);
        thread t2(printNumbers);
        t1.join();
        t2.join();
        return 0;
    }
    """,
    """
    #include <iostream>
    using namespace std;
    class Singleton {
    private:
        static Singleton* instance;
        Singleton() {}
    public:
        static Singleton* getInstance() {
            if (!instance) {
                instance = new Singleton();
            }
            return instance;
        }
    };
    Singleton* Singleton::instance = nullptr;
    int main() {
        Singleton* s1 = Singleton::getInstance();
        Singleton* s2 = Singleton::getInstance();
        cout << (s1 == s2) << endl;  // Output: 1 (true)
        return 0;
    }
    """
]

# Define the structured prompt
problem_prompt = PromptTemplate(
    input_variables=["idx", "topic", "solution"],
    template=(
        "You are a C++ programming instructor. Here is the topic and solution from a student.\n\n"
        "Topic {idx}: {topic}\n\n"
        "Student Solution:\n{solution}\n\n"
        "Your task is to provide the following details:\n"
        "1. Problem description and problem statement (summarize).\n"
        "2. Detailed evaluation of the student's solution based on correct use of OOP principles (30 points), "
        "code readability and comments (20 points), handling of edge cases (20 points), and correctness of the solution (30 points).\n"
        "3. Provide constructive feedback for the student, highlighting strengths and areas for improvement."
    )
)

# Function to generate output using Cohere's LLM (No-RAG approach)
def generate_output_with_cohere_no_rag(topics, student_solutions):
    output = ""

    for idx, (topic, solution) in enumerate(zip(topics, student_solutions)):
        # Create the problem chain using the prompt template
        problem_chain = LLMChain(llm=cohere_llm, prompt=problem_prompt)

        # Generate the output using the chain
        result = problem_chain.run(idx=idx, topic=topic, solution=solution)

        # Append the generated text to the output, including student solution
        output += f"\n_________________topic number {idx}___________________\n\n"
        output += f"Topic: {topic}\n"
        output += f"Student Solution:\n{solution}\n\n"
        output += f"Generated Evaluation and Feedback:\n{result}\n"

    return output

# Generate the output
output = generate_output_with_cohere_no_rag(sample_topics, sample_solutions)

# Write the output to a file
def write_output_to_file(output, filename="cpp_oop_norag.txt"):
    with open(filename, "w") as file:
        file.write(output)

write_output_to_file(output)
